In [53]:
from selenium import webdriver
import pandas as pd
import openpyxl as op
import time
import re
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
get_number = re.compile('.*(?=\n)')
get_percent = re.compile('.*(?=%)')
get_url_without_slash = re.compile(".*[0-9a-z]")
password = !echo %password%

In [2]:
def login():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")
    #아이디입력
    driver.find_element_by_css_selector('#Email').send_keys('taehun9312@gmail.com')
    driver.find_element_by_css_selector('#next').click()
    time.sleep(1)
    #비밀번호입력
    driver.find_element_by_css_selector('#Passwd').send_keys(password[0])
    driver.find_element_by_css_selector('#signIn').click()
    
def s_to_i(value):
    value = value.replace(",","")
    value = int(value)
    return value

def check(compared_value):
    if type(compared_value) == list:
        for value in compared_value:
            if value in global_value: raise ValueError
    elif compared_value in global_value: raise ValueError
    else: print("No_duplication")
        
def get_pure_url(url):
    splited_url = url.split("/%")
    url = splited_url[0]
    url = get_url_without_slash.findall(url)[0]
    return url

def get_number_item(unprocessed_number):
    processed_number = unprocessed_number.split("(")[0]
    return s_to_i(processed_number)

        
def calculate_date(origin_date, interval, multiple_number):
    date_set = _get_date(origin_date)
    d1 = date(date_set["year"], date_set["month"], date_set["day"])
    if interval == "month":
        d2 = d1 + relativedelta(months=+1*multiple_number)
    else:
        d2 = d1 + timedelta(days = interval*multiple_number)
    next_date = _to_date(d2)
    return next_date

def _get_date(date):
    date = str(date)
    year = "20" + date[0:2]
    month = date[2:4]
    day = date[4:6]
    if month[0] == "0":
        month = month.replace("0","")
    return {"year": int(year), "month": int(month), "day": int(day)}

def _to_date(date):
    year = str(date.year)[2:4]
    if date.month < 10:
        month = "0" + str(date.month)
    else:
        month = str(date.month)
    if date.day < 10:
        day = "0" + str(date.day)
    else:    
        day = str(date.day)
    return year+month+day

In [64]:
class GA:
    
    def __init__(self, url):
        global start
        global end 
        self.df = pd.DataFrame()
        self.date = "/%3F_u.date00%3D20{start}%26_u.date01%3D20{end}/".format(start= start, end=end)
        self.original_url = url
        self.url = get_pure_url(url) + self.date
        self.start = start
        self.end = end
        self.retry = 40
        self.number = 1
        self.df_start = str(start)
        self.df_end = str(end)
        self.dic_values = {}
        driver.get(self.url)
        
    def set_date(self, interval, number):
        self.interval = interval
        self.number = number

        
    def set_segment_url(self, seg):
        self.url = get_url_without_slash.findall(self.url)[0] + seg
        #"%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ"
        
    def set_tab(self, n_tab):
        self.tab = n_tab
        
    def set_value(self, n_value):
        if type(n_value) != list:
            self.value = [n_value]
        else:
            self.value = n_value
        
    def set_retry(self, n):
        self.retry = n
    
    def go_changed_url(self, n_try):
        start = calculate_date(self.start, self.interval, n_try)
        end = calculate_date(self.end, self.interval, n_try)
        self.df_start = str(start)
        self.df_end = str(end)
        self.date =  "/%3F_u.date00%3D20{start}%26_u.date01%3D20{end}/".format(start= start, end=end)
        self.url = get_pure_url(self.original_url) + self.date
        driver.get(self.url)
        
    def get(self, numbers):
        for n_try in range(1, self.number+1):
            if type(numbers) == int:
                value = self._come_to_me(numbers)
                return value
            else:
                for n in numbers:
                    self.dic_values = self._fill_dic_values(n, self.dic_values)
            for title, value in self.dic_values.items():
                    self.df.loc[self.df_start+" - "+self.df_end, title] = value
            if self.number > 1 and n_try < self.number: 
                    self.dic_values = {}
                    self.go_changed_url(n_try)  
        self.df = self.df.transpose()
 

    def _fill_dic_values(self, numbers, dic_values):
            n = numbers
            value = self._come_to_me(n)
            titles = self._get_title(self.value[n-1])
            
            if type(value) != list:
                if type(self.value[n-1]) != list:
                    #title 1, value 1
                    title = titles[0]
                    if title in dic_values:
                        title = title + str(n)
                    dic_values[title] = value
                else:
                    #titile more, value 1
                    nth = self.value[n-1][1]
                    title = titles[nth]
                    if title in dic_values:
                        title = title + str(n)
                    dic_values[title] = value
            else:
                #title more, value more
                for i, v in enumerate(value):
                    title = titles[i]
                    if title in dic_values:
                        title = title + str(n)
                    dic_values[title] = v 
            return dic_values
    
    def _come_to_me(self, n):
        global global_value
        error_check = "on"
        self._click_tab(n)

        if type(self.value) == list:
            value_spec = self.value[n-1]
        else:
            value_spec = self.value

        if type(value_spec) == list: 
            number = value_spec[0] 
            nth = value_spec[1]
        else: 
            number = value_spec
            nth = value_spec

        if type(value_spec) == list or value_spec == 1:
            value = self._get_value(nth, error_check)
            error_check = "off"
            value_for_compare = self._get_values(number, error_check)
        else:
            value_for_compare = self._get_values(number, error_check)
            value = value_for_compare
        global_value = value_for_compare
        return value

    def _get_title(self, n):
        arr_title = []
        unprocessed_titles = driver.find_elements_by_css_selector("._GArO")
        for title in unprocessed_titles:
            arr_title.append(title.text)
        return arr_title
    
    def get_percent(self, n):
        arr_value = driver.find_elements_by_css_selector("._GAj8")
        percent = arr_value[n-1].text
        processed_percent = float(get_percent.findall(percent)[0])
        print(processed_percent)
        return processed_percent
        
            
    def get_item(self, name, nth):
        for i in range(0, self.retry):
            try:
                items = driver.find_elements_by_css_selector("._GApu")
                dataset = driver.find_elements_by_css_selector("._GAtjb")
                for item in items:
                    if item.text == name:
                        value = get_number_item(dataset[i*2+nth-1].text)
                        check(value)
                        print("It's ok")
                        arr_data = []
                        for data in dataset:
                            arr_data.append(data.text)
                        global_value = arr_data
                        print(value)
                        return value
                time.sleep(1)
            except:
                time.sleep(4)
                print("It's not ok")
                continue
                        

    #n번째 값 가져오기
    def _get_value(self, n, error_check):
        for i in range(0, self.retry):
            try:
                print(i)
                value = driver.find_element_by_css_selector(".ID-stage{n}-top._GAhFb > ._GAXm > ._GAoCb".format(n = n)).text
                if n != 1: value = get_number.findall(value)[0]
                value = s_to_i(value)
                print("value is", value)
                if error_check == "on":
                    check(value)
                print("It's ok")
                break
            except:
                time.sleep(4)
                print("It's not ok")
                continue
        print(value)
        return value
    
    #n개의 값 가져오기
    def _get_values(self, range_n, error_check):
        arr_value = []
        for n in range(1, range_n+1):
            value = self._get_value(n, error_check)
            arr_value.append(value)
        print(arr_value)
        return arr_value

    #커스텀 리포트 내 탭
    def _click_tab(self, n):
        if n == 1:
            driver.get(self.url)
        else:
            for i in range(0, self.retry):
                try:
                    driver.find_elements_by_css_selector("._GAit._GAl")[n-1].click()
                    break
                except:
                    time.sleep(3)
                    print("It retry to click")
                    continue

In [65]:
# excel = op.load_workbook(r"C:\Users\joseph_th\Desktop\Dashboard\Dashboard_for_GA.xlsx")
# s1 = excel["여행 Traffic overview"]["AC"]
# s2 = excel["여행 딜"]["W"]
# s3 = excel["실시간항공"]["Z"]
# s4 = excel["실시간호텔"]["U"]
# s5 = excel["APP 홈 기여"]["S"]

In [66]:
driver = webdriver.Chrome(executable_path=r"C:\Users\joseph_th\Downloads\chromedriver_win32\chromedriver")
login()
global_value = []
start = 170301
end = 170310

In [12]:
# excel.save(r"C:\Users\joseph_th\Desktop\Dashboard\Dashboard_for_GA_result.xlsx")

In [17]:
# #국내, 해외, 항공호텔, 전체카테고리
# And_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/XAzFnmCtTl6SM1Cmc9OVNw/a10587421w80067081p102782120/")
# And_1.set_tab(5)
# And_1.set_value([1,1,1,1])
# And_1.get([1,2,3,4])

# #티몬 전체 카테고리
# And_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/gmdw7utOS6OyMdXRs_jizg/a10587421w80067081p102782120/")
# And_2.set_tab(1)
# And_2.set_value(1)
# And_2.get([1])

# #국내, 해외, 여행전체 딜
# And_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/c9agY1srRx2PaD8JAkK0yw/a10587421w80067081p102782120/")
# And_3.set_tab(3)
# And_3.set_value([3,3,3])
# And_3.get([1,2,3])


# And_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ")
# And_3.set_value([[3,3],[3,3],[3,3]])
# And_3.get([1,2,3])


# #항공검색 해외, 국내, 해외->호텔메인, 해외->국내항공메인
# And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5fyYOU-QTN6tQSwRlObkUw/a10587421w80067081p102782120/%3F_u.date00%3D20170101%26_u.date01%3D20170301%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# And_4.set_tab(4)
# And_4.set_value([2,2,[2,2],[2,2]])
# And_4.get([1,2,3,4])

# #항공예약 클릭 국내, 해외 
# And_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/_yDrAZzXTaeMOGlYkwqqbw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D%26_r.tabId%3D6028/")
# And_5.set_tab(2)
# And_5.set_value([1,1])
# And_5.get([1,2])

# #부킹닷컴 큐레이션, 호텔배너, 주말특가, 검색모듈
# And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
# And_6.set_tab(4)
# And_6.set_value([2,1,2,2])
# curation = And_6.get([1,2,3,4])

# #부킹닷컴 이벤트페이지
# And_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/qWyiPek-Tsqgkok-rZsIjA/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
# And_7.set_tab(1)
# And_7.set_value(2)
# And_7.get([1])

# #홈 -> 딜상세 국내, 해외, 카테고리숏컷
# And_8 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/4rkbjB7zTGmgcRAlJaVzrg/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
# And_8.set_tab(3)
# And_8.set_value([2,[2,2],1])
# And_8.get([1,2,3])

# #홈 -> 콜렉션(국내), 해외
# And_9 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/lUOk-ms-R966phrt04_yfA/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
# And_9.set_tab(2)
# And_9.set_value([2,[2,2]])
# And_9.get([1,2])


# #여행탭 -> 카테고리, 항공검색결과
# And_11 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/rUsbr2MuTou-MkNzI27mfw/a10587421w80067081p102782120/")
# And_11.set_tab(3)
# And_11.set_value([2,[2,2]])
# And_11.get([1,2])

In [9]:
# #국내, 해외, 항공호텔, 전체카테고리
# iOS_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/f6fCST7zQwalCMN1dpSuPw/a10587421w80067081p88823264/")
# iOS_1.set_tab(5)
# iOS_1.set_value([1,1,1,1])
# iOS_1.get([1,2,3,4])


# #티몬 전체 카테고리
# iOS_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p29BV_8iRc2OovJCvhcg_Q/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402/")
# iOS_2.set_tab(1)
# iOS_2.set_value(1)
# iOS_2.get([1])

# #국내딜, 해외딜, 전체딜
# iOS_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/A9m3wyZgQTmxJrO3lQ6UMw/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402%26_r.tabId%3D4646%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# iOS_3.set_tab(3)
# iOS_3.set_value([3,3,3])
# iOS_3.get([1,2,3])

# iOS_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ")
# iOS_3.set_value([[3,3],[3,3],[3,3]])
# iOS_3.get([1,2,3])

# #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
# iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/2K60xdJeT2isKsPe4tgpEQ/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170320/")
# iOS_4.set_tab(4)
# iOS_4.set_value([2,2,[2,2],[2,2]])
# iOS_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# iOS_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/btj6gUJmQ2-NZsSB1vB3tw/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170331%26_.useg%3D/")
# iOS_5.set_tab(2)
# iOS_5.set_value([1,1])
# iOS_5.get([1,2])

# #부킹닷컴 큐레이션, 호텔배너, 주말특가, 해외호텔검색모듈
# iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/fiA5QbsjQAeJxkmaYUFJEQ/a10587421w80067081p88823264/")
# iOS_6.set_tab(4)
# iOS_6.set_value([2,1,2,2])
# iOS_6.get([1,2,3,4])

# #부킹닷컴 이벤트
# iOS_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/wwJNU8e1SheBWaYfk-Bulw/a10587421w80067081p88823264/")
# iOS_7.set_tab(1)
# iOS_7.set_value(2)
# iOS_7.get([1])

# #홈 -> 딜상세(국내), 해외, 여행카테고리숏컷
# iOS_8 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/GDfZCJv9RSC7dYyccZZ1lA/a10587421w80067081p88823264/")
# iOS_8.set_tab(3)
# iOS_8.set_value([2,[2,2],1])
# iOS_8.get([1])

# #홈 -> 콜렉션(국내), 해외
# iOS_9 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/9rdCrGeYTaCIuaCGnfLoRQ/a10587421w80067081p88823264/")
# iOS_9.set_tab(2)
# iOS_9.set_value([2,[2,2]])
# iOS_9.get([1,2])

In [10]:
# #국내카테고리, 해외, 항공호텔, 전체
# Mo_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/V-T6iNbTQWi5USC2dSi5tA/a10587421w80067081p89357101/")
# Mo_1.set_tab(5)
# Mo_1.set_value([1,1,1,2])
# Mo_1.get([1,2,3,4])

# #티몬 홈 전체
# Mo_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/zoovm6xETN-Cd6JF9s2_hQ/a10587421w80067081p89357101/")
# Mo_2.set_tab(1)
# Mo_2.set_value(1)
# Mo_2.get([1])

# #여행딜 전체, 국내, 해외
# Mo_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/upJUz96ERVGUIE2WWd8Tgg/a10587421w80067081p89357101/")
# Mo_3.set_tab(3)
# Mo_3.set_value([3,3,3])
# Mo_3.get([1,2,3])
# Mo_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ")
# Mo_3.set_value([[3,3],[3,3],[3,3]])
# Mo_3.get([1,2,3])

# #항공검색 해외, 국내, 해외->호텔, 해외->국내
# Mo_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/7aMTsZECQu2GVnxwp3IXRg/a10587421w80067081p89357101/")
# Mo_4.set_tab(4)
# Mo_4.set_value([2,2,[2,2],[2,2]])
# Mo_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# Mo_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Shs74NpmR1Wcy5isHt_ZGA/a10587421w80067081p89357101/")
# Mo_5.set_tab(2)
# Mo_5.set_value([1,1])
# Mo_5.get([1,2])

# #부킹닷컴 큐레이션, 호텔메인
# Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/")
# Mo_6.set_tab(2)
# Mo_6.set_value([2,2])
# Mo_6.get([1,2])

# #부킹닷컴 이벤트
# Mo_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/gY8HkfiiQaqkTZOucG8YFQ/a10587421w80067081p89357101/")
# Mo_7.set_tab(1)
# Mo_7.set_value(2)
# Mo_7.get([1])

In [11]:
# # 카테고리 국내, 해외, 항공호텔, 전체
# # PC_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/RFQ6OlJDT5-6MMKnjtJkcA/a10587421w80067081p84826225/")
# # PC_1.set_tab(4)
# # PC_1.set_value([1,1,[2,2],2])

# # 티몬 전체 카테고리
# PC_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ZcNefO8cTRe75IKFbPcCWw/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_2.set_tab(1)
# PC_2.set_value(1)
# PC_2.get([1])

# # 여행 전체, 국내, 해외딜
# PC_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hRoEL4w4RCyYpcmhDbvgWw/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_3.set_tab(3)
# PC_3.set_value([3,3,3])
# PC_3.get([1,2,3])
# PC_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ")
# PC_3.set_value([[3,3],[3,3],[3,3]])
# PC_3.get([1,2,3])

# # 항공검색 국내, 해외
# PC_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/KkWuVj5-SCOH7ddtrf4bOA/a10587421w80067081p84826225/")
# PC_4.set_tab(2)
# PC_4.set_value([2,2])
# PC_4.get([1,2])

# # 예약클릭 국내, 해외
# PC_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/YspUfeZLQ7KyhFzRxehRcQ/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_5.set_tab(2)
# PC_5.set_value([2,2])
# PC_5.get([1,2])

# # 부킹닷컴 큐레이션, 해외호텔 카테고리, 주말특가
# PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_6.set_tab(3)
# PC_6.set_value([2,1,2])
# PC_6.get([1,2,3])

# # 부킹닷컴 이벤트
# PC_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/K68VmWT0QxSxn2j0wohK4A/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_7.set_tab(1)
# PC_7.set_value(2)
# PC_7.get([1])

In [34]:
# #######월간 여행딜

# #국내, 해외, 여행전체 딜
# And_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/c9agY1srRx2PaD8JAkK0yw/a10587421w80067081p102782120/")
# And_3.set_tab(3)
# And_3.set_value([3,3,3])
# And_3.get([1,2,3])

# #국내딜, 해외딜, 전체딜
# iOS_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/A9m3wyZgQTmxJrO3lQ6UMw/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402%26_r.tabId%3D4646%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# iOS_3.set_tab(3)
# iOS_3.set_value([3,3,3])
# iOS_3.get([1,2,3])

# iOS_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ")
# iOS_3.set_value([[3,3],[3,3],[3,3]])
# iOS_3.get([1,2,3])

# #여행딜 전체, 국내, 해외
# Mo_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/upJUz96ERVGUIE2WWd8Tgg/a10587421w80067081p89357101/")
# Mo_3.set_tab(3)
# Mo_3.set_value([3,3,3])
# Mo_3.get([1,2,3])
# Mo_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ")
# Mo_3.set_value([[3,3],[3,3],[3,3]])
# Mo_3.get([1,2,3])

# # 여행 전체, 국내, 해외딜
# PC_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hRoEL4w4RCyYpcmhDbvgWw/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_3.set_tab(3)
# PC_3.set_value([3,3,3])
# PC_3.get([1,2,3])
# PC_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ")
# PC_3.set_value([[3,3],[3,3],[3,3]])
# PC_3.get([1,2,3])

In [72]:
# # #국내, 해외, 항공호텔, 전체카테고리
# df1 = pd.concat([And_1.df, iOS_1.df, Mo_1.df, PC_1.df])

# # #티몬 전체 카테고리
# df2 = pd.concat([And_2.df, iOS_2.df, Mo_2.df, PC_2.df])

# # #국내딜, 해외딜, 전체딜
# df3 = pd.concat([And_3.df, iOS_3.df, Mo_3.df, PC_3.df])

# # #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
# df4 = pd.concat([And_4.df, iOS_4.df, Mo_4.df, PC_4.df])

# # #항공예약클릭 국내, 해외
# df5 = pd.concat([And_5.df, iOS_5.df, Mo_5.df, PC_5.df])

# # #부킹닷컴 큐레이션, 호텔배너, 주말특가, 해외호텔검색모듈
# df6 = pd.concat([And_6.df, iOS_6.df, Mo_6.df, PC_6.df])

# # #부킹닷컴 이벤트
# df7 = pd.concat([And_7.df, iOS_7.df, Mo_7.df, PC_7.df])

# # #홈 -> 딜상세(국내), 해외, 여행카테고리숏컷
# df8 = pd.concat([And_8.df, iOS_8.df, Mo_8.df, PC_8.df])

# # #홈 -> 콜렉션(국내), 해외
# df9 = pd.concat([And_9.df, iOS_9.df, Mo_9.df, PC_9.df])


In [73]:
# #######월간항공

# #항공검색 해외, 국내, 해외->호텔메인, 해외->국내항공메인
# And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5fyYOU-QTN6tQSwRlObkUw/a10587421w80067081p102782120/%3F_u.date00%3D20170101%26_u.date01%3D20170301%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# And_4.set_tab(4)
# And_4.set_value([2,2,[2,2],[2,2]])
# And_4.get([1,2,3,4])

# #항공예약 클릭 국내, 해외 
# And_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/_yDrAZzXTaeMOGlYkwqqbw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D%26_r.tabId%3D6028/")
# And_5.set_tab(2)
# And_5.set_value([1,1])
# And_5.get([1,2])

# #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
# iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/2K60xdJeT2isKsPe4tgpEQ/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170320/")
# iOS_4.set_tab(4)
# iOS_4.set_value([2,2,[2,2],[2,2]])
# iOS_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# iOS_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/btj6gUJmQ2-NZsSB1vB3tw/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170331%26_.useg%3D/")
# iOS_5.set_tab(2)
# iOS_5.set_value([1,1])
# iOS_5.get([1,2])

# #항공검색 해외, 국내, 해외->호텔, 해외->국내
# Mo_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/7aMTsZECQu2GVnxwp3IXRg/a10587421w80067081p89357101/")
# Mo_4.set_tab(4)
# Mo_4.set_value([2,2,[2,2],[2,2]])
# Mo_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# Mo_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Shs74NpmR1Wcy5isHt_ZGA/a10587421w80067081p89357101/")
# Mo_5.set_tab(2)
# Mo_5.set_value([1,1])
# Mo_5.get([1,2])

# # 항공검색 국내, 해외
# PC_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/KkWuVj5-SCOH7ddtrf4bOA/a10587421w80067081p84826225/")
# PC_4.set_tab(2)
# PC_4.set_value([2,2])
# PC_4.get([1,2])

# # 예약클릭 국내, 해외
# PC_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/YspUfeZLQ7KyhFzRxehRcQ/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_5.set_tab(2)
# PC_5.set_value([2,2])
# PC_5.get([1,2])